#PREDICT THE BURNED AREA OF FOREST FIRES WITH NEURAL NETWORKS


In [19]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Layer, Lambda

In [20]:
df = pd.read_csv("forestfires.csv")
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [21]:
#As dummy variables are already created, we will remove the month and also day columns
df.drop(["month", "day"], axis = 1, inplace = True)

In [22]:
df["area"] = np.where(df["area"] > 50, 1, 0)

In [23]:
df["area"].value_counts()

0    493
1     24
Name: area, dtype: int64

In [24]:
df.isnull().sum()

FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [25]:
df.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,...,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,0.046422,0.164410,...,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,0.210600,0.371006,...,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
#Normalization being done.
def norm_func(i):
     x = (i - i.min()) / (i.max() -	i.min())
     return (x)

In [27]:
predictors = df.iloc[ :, 0:8]
target = df.iloc[ :, 8]

predictors1 = norm_func(predictors)
#data = pd.concat([predictors1,target],axis=1)

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(predictors1, target, test_size = 0.2, stratify = target)

In [29]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1, len(hidden_dim) - 1):
        if (i == 1):
            model.add(Dense(hidden_dim[i], input_dim = hidden_dim[0], activation = "relu"))
        else:
            model.add(Dense(hidden_dim[i], activation = "relu"))
    model.add(Dense(hidden_dim[-1], kernel_initializer = "normal", activation = "sigmoid"))
    model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ["accuracy"])
    return model  

In [30]:
#y_train = pd.DataFrame(y_train)
    
first_model = prep_model([8, 50, 40, 20, 1])
first_model.fit(np.array(x_train), np.array(y_train), epochs = 750)
pred_train = first_model.predict(np.array(x_train))

Epoch 1/750
13/13 [==============================] - 1s 2ms/step - loss: 0.6551 - accuracy: 0.8959
Epoch 2/750
13/13 [==============================] - 0s 2ms/step - loss: 0.5468 - accuracy: 0.9540
Epoch 3/750
13/13 [==============================] - 0s 2ms/step - loss: 0.3997 - accuracy: 0.9540
Epoch 4/750
13/13 [==============================] - 0s 2ms/step - loss: 0.2709 - accuracy: 0.9540
Epoch 5/750
13/13 [==============================] - 0s 2ms/step - loss: 0.2105 - accuracy: 0.9540
Epoch 6/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1963 - accuracy: 0.9540
Epoch 7/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1949 - accuracy: 0.9540
Epoch 8/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1949 - accuracy: 0.9540
Epoch 9/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1938 - accuracy: 0.9540
Epoch 10/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1924 - accuracy: 0.9540
Epoch 11/

In [31]:
#Converting the predicted values to series 
pred_train = pd.Series([i[0] for i in pred_train])

size = ["small", "large"]
pred_train_class = pd.Series(["small"]*413)
pred_train_class[[i > 0.5 for i in pred_train]] = "large"

train = pd.concat([x_train, y_train], axis = 1)
train["area"].value_counts()

0    394
1     19
Name: area, dtype: int64

In [32]:
# Cheking with prediction for training data
from sklearn.metrics import confusion_matrix
train["original_class"] = "small"
train.loc[train["area"] == 1, "original_class"] = "large"
train.original_class.value_counts()
confusion_matrix(pred_train_class, train["original_class"])
np.mean(pred_train_class == pd.Series(train["original_class"]).reset_index(drop = True)) #98.54%
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop = True))


original_class,large,small
row_0,,
large,15,1
small,4,393


In [33]:
#For test data
pred_test = first_model.predict(np.array(x_test))
pred_test = pd.Series([i[0] for i in pred_test])
pred_test_class = pd.Series(["small"]*104)
pred_test_class[[i>0.5 for i in pred_test]] = "large"
test =pd.concat([x_test, y_test], axis = 1)
test["original_class"] = "small"
test.loc[test["area"] == 1, "original_class"] = "large"


4/4 [==============================] - 0s 3ms/step


In [34]:

test["original_class"].value_counts()

small    99
large     5
Name: original_class, dtype: int64

In [35]:
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop = True)) # 95.19%

0.9326923076923077

In [36]:
confusion_matrix(pred_test_class,test["original_class"])
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop = True))

original_class,large,small
row_0,,
large,0,2
small,5,97
